# MODULE INSTALLATIONS

In [3]:
import datetime
import traceback
import time
import requests
import pandas as pd
from sklearn.model_selection import train_test_split

# INITIALIZE THE WORKSPACE

In [5]:
MODEL_NAME = "all_leagues"
MODEL_VERSION = "v1.0"

# FUNCTION DEFINATIONS

In [9]:
# INDICATOR PERIODS
MA_RSI_PERIOD = 14
BOLLINGER_PERIOD = 20
MACD_SIGNAL_PERIOD = 9
MACD_SHORT_PERIOD = 12
MACD_LONG_PERIOD = 26
BOLLINGER_N_STD = 2


# FINANCIAL INDICATORS
def rsi(data, window = MA_RSI_PERIOD, modify = True):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))

    if modify:
      data["rsi"] = rsi
      return data
    else:
      return rsi


def sma(data, window = MA_RSI_PERIOD, modify = True):
    _sma = data['close'].rolling(window=window).mean()

    if modify:
      data["sma"] = _sma
      return data
    else:
      return _sma


def ema(data, span = MA_RSI_PERIOD, modify = True):
    _ema = data['close'].ewm(span=span, adjust=False).mean()

    if modify:
      data["ema"] = _ema
      return data
    else:
      return _ema


def stochastic_oscillator(data, window = MA_RSI_PERIOD, modify = True):
    lowest_low = data['low'].rolling(window=window).min()
    highest_high = data['high'].rolling(window=window).max()
    _stochastic = ((data['close'] - lowest_low) / (highest_high - lowest_low)) * 100

    if modify:
      data["stochastic"] = _stochastic
      return data
    else:
      return _stochastic


def bollinger_bands(data, window = BOLLINGER_PERIOD, num_std = 2, modify = True):
    rolling_mean = data['close'].rolling(window=window).mean()
    rolling_std = data['close'].rolling(window=window).std()

    _lower_band = rolling_mean - (rolling_std * num_std)
    _upper_band = rolling_mean + (rolling_std * num_std)

    if modify:
      data["bollinger_lower"] = _lower_band
      data["bollinger_upper"] = _upper_band
      return data
    else:
      return _lower_band, _upper_band


def macd(data, short_window = MACD_SHORT_PERIOD, long_window = MACD_LONG_PERIOD, signal_window = MACD_SIGNAL_PERIOD, modify = True):
    _ema_short = ema(data, span=short_window, modify=False)
    _ema_long = ema(data, span=long_window, modify=False)

    macd_line = _ema_short - _ema_long
    signal_line = macd_line.rolling(window=signal_window).mean()

    if modify:
      data["macd"] = macd_line
      data["macd_signal"] = signal_line
      return data
    else:
      return macd_line, signal_line


def to_lowercase_columns(data):
    data.columns = [column.replace(" ", "_").lower() for column in data.columns]
    return data


def append_technical_indicators(data, add_outcomes = True):
  # data = to_lowercase_columns(data)
  data = rsi(data)
  data = sma(data)
  data = ema(data)
  data = stochastic_oscillator(data)
  data = bollinger_bands(data)
  data = macd(data)

  data.index = pd.to_datetime(data.pop("timestamp"))
  data.index.names = ["timestamp"]
  data = data.sort_values("timestamp")

  data["hour"] = data.index.hour
  data["day_of_week"] = data.index.dayofweek
  data = data.dropna()

  # OUTCOMES OF THE TICKER
  if add_outcomes:
    data["target"] = (data.close > data.open).apply(lambda x: 1 if x > 0 else 0)
  return data.astype(float)

# DATA SCRAPING

In [14]:
"""Gets head to head stats for a game for the previous 10 matches."""
def get_head2head(match_id):
  endpoint = f"https://api.football-data.org/v4/matches/{match_id}/head2head?limit=5"

  request_id = refresh_expired_keys(api_keys)
  if request_id:
    response = requests.get(endpoint,
                      headers={'X-Unfold-Lineups': "true",
                              'X-Unfold-Goals': "true",
                              'X-Auth-Token': request_id})

    if response.status_code == 200:
      return response.json().get("aggregates", dict())
    else:
      return dict()


def refresh_expired_keys(api_keys):
    current_time = time.time()
    found_key = False

    while found_key == False:
      for i, key_info in enumerate(api_keys):
        last_request_time = key_info[2]
        if current_time - last_request_time >= 60:
            # Refresh the key if it has expired
            api_keys[i][2] = current_time
            api_keys[i][1] = 0

      # Find a key with available requests less than 10
      for key_info in api_keys:
          if key_info[1] < 10:
              found_key = True
              key_info[1] += 1
              print(key_info[0])
              return key_info[0]  # Return the API key


def get_matches(api_key):
  competitions = ["PL"] #, "ELC", "EC", "FL1", "BL1", "SA", "DED", "CLI"
  response = requests.get(f"https://api.football-data.org/v4/matches?competitions={','.join(competitions)}&dateTo={today.strftime('%Y-%m-%d')}&dateFrom={past_ten_days.strftime('%Y-%m-%d')}",
                    headers={'X-Unfold-Lineups': "true",
                             'X-Unfold-Goals': "true",
                             'X-Auth-Token': api_key})
  if response.status_code == 200:
    return response.json()["matches"]
  else:
    return []
  

# Initialize API keys with 0 timestamp
api_keys = [
   # KEY, No. REQUESTS MADE, LAST REQ. TIMESTAMP
   ["9ff4b5192f4b4272855493db973d8c61", 0, 0],
   ["a151d6c3003c4f838ffa519bea3315a1", 0, 0],
   ["618a74d05cfb4401b7a66d4ed4c3cae5", 0, 0],
   ["f8024b296b70487c852eff7a353d8f85", 0, 0],
   ["024b012d8df24b3b9ee70863633e0fcc", 0, 0],
   ["1a65d4d0d7cc4f5893b438b4092da912", 0, 0],
   ["c872982f9423441fbfab53d52c8558b4", 0, 0],
   ["bd8c94c0ce6e435d8ddc6f3d0d2ba0aa", 0, 0],
   ["5433c517fdf0408da4578b14bd95b74b", 0, 0],
   ["be9ee655916a465895ccd17f3026422a", 0, 0],
   ["61659ea176ba454f98a421ec6facaa3d", 0, 0],
   ["9cd7992817f04d5abfbe6ffa02624d6d", 0, 0],
   ["3fab499119a547f1abf56b83db88ca1b", 0, 0],
   ["ec75a3fc37c1403db385c598e3cef779", 0, 0],
   ["2bb6d52219664b18b5efde63d075a03b", 0, 0],
   ["9b648751e85443f1bde592d4b337b7e1", 0, 0],
   ["2819b51d87214f98a21bbff55bb6b4d4", 0, 0],
   ["611db8b59ba244d6bab7fc71b8d23ac9", 0, 0],
   ["cce392b8fca641f3a41361756a3cd36b", 0, 0],
   ["542f193f142d42a5845ffda13c60b705", 0, 0],
   ["8ff44d4142974404875bb5ecb71eba25", 0, 0],
   ["ba93366e933a43e4b2f0ffe2f32c17a7", 0, 0],
   ["3aa9fa11915b4ff595ccd91cc75834fd", 0, 0],
   ["92eecef75959449e84abb7559a2f2180", 0, 0]
]

In [ ]:
# Control structure loop variables
today = datetime.datetime.now()
past_ten_days = today - datetime.timedelta(days=10)
print(today, past_ten_days)

COLUMNS = ["id", "day", "month", "year", "weekday", "hour", "minute", "matchday", "status", "leagueName", "leagueId", "leagueType", "stage", "homeName", "homeId", "awayName", "awayId", "scoreHomeHt", 
                    "scoreAwayHt", "scoreHomeFt", "scoreAwayFt"] #, "agHomeWins", "agHomeLosses", "agDraws", "agHomeWins", "agHomeLosses", "agTotalGoals", "noOfAgMatches"
TOTAL_DATA_REQUIRED = 100_000
request_count = 0
data = []


while len(data) < TOTAL_DATA_REQUIRED:
    try:
        selected_key = refresh_expired_keys(api_keys)
        if selected_key:
            matches, last_date = get_matches(selected_key)
            print("Received matches:", len(matches))
            for match in matches:
                # head2head = get_head2head(match["id"])
                matchdate = datetime.datetime.strptime(match["utcDate"], "%Y-%m-%dT%H:%M:%SZ")

                data.append([
                    match["id"], # id
                    matchdate.day, # id
                    matchdate.month, # month
                    matchdate.year, # id
                    matchdate.isoweekday(), # dayofweek
                    matchdate.hour, # hour
                    matchdate.minute, # hour
                    match["matchday"], # status
                    match["status"], # status
                    match["competition"]["name"], # leagueName
                    match["competition"]["id"], # leagueId
                    match["competition"]["type"], # leagueType
                    match["stage"], # stage
                    match["homeTeam"]["shortName"], # homeName
                    match["homeTeam"]["id"], # homeId
                    match["awayTeam"]["shortName"], # awayName
                    match["awayTeam"]["id"], # awayId
                    match["score"]["halfTime"]["home"], # scoreHomeHt
                    match["score"]["halfTime"]["away"], # scoreAwayHt
                    match["score"]["fullTime"]["home"], # scoreHomeFt
                    match["score"]["fullTime"]["away"], # scoreAwayFt
                ])
                            
            # Simulate making a request with the selected key
            for key_info in api_keys:
                if key_info[0] == selected_key:
                    key_info[1] += 1
                    key_info[2] = time.time()  # Update last request time
                    request_count += 1
                    
                    print("Request count:", request_count)
                    print("Matches count:", len(data))
                    break
            
            today = past_ten_days
            past_ten_days = today - datetime.timedelta(days=10)
            print(today, past_ten_days)
        else:
            print("No API Key with available requests less than 10. Waiting 60 seconds...")
            time.sleep(60)  # Wait for 60 seconds before trying again
            # Reset the api keys after this point
            api_keys = [[api_key[0], 0, 0] for api_key in api_keys]
            
            today = past_ten_days
            past_ten_days = today - datetime.timedelta(days=10)
            print(today, past_ten_days)

    except KeyboardInterrupt:
        # Incase something goes wrong with the above script, save current progress.
        df = pd.DataFrame(data, columns=COLUMNS)
        df.to_csv(f"../relative_datasets/raw/{MODEL_NAME}-{MODEL_VERSION}_{request_count}.csv", index=False)
        break
    except:
        print(traceback.format_exc())
        print("Today:", today, "Past 10 days:", past_ten_days)
        # Await the next 60 seconds to be allowed to request more data, incase limit has been reached.
        time.sleep(60)
    
# Incase something goes wrong with the above script, save current progress.
df = pd.DataFrame(data, columns=COLUMNS)
df.to_csv(f"../relative_datasets/raw/{MODEL_NAME}-{MODEL_VERSION}_{request_count}.csv", index=False)
print("Total requests made:", request_count)

In [13]:
get_head2head(494875)

{'numberOfMatches': 1,
 'totalGoals': 5,
 'homeTeam': {'id': 5184,
  'name': 'CA Peñarol',
  'wins': 0,
  'draws': 0,
  'losses': 1},
 'awayTeam': {'id': 1766,
  'name': 'CA Mineiro',
  'wins': 1,
  'draws': 0,
  'losses': 0}}

# DATA ENGENEERING

In [ ]:
DATAFILE_PATH = "../relative_datasets/raw/all_leagues-v1.0_900.csv"
dataframe = pd.read_csv(f"{DATAFILE_PATH}")
dataframe = dataframe.dropna().drop_duplicates().reset_index(drop=True)
dataframe

In [ ]:
try:
    # Add head2head stats for better comparability
    copy = dataframe.copy()
    copy2 = pd.DataFrame([], columns=[*copy.columns, "h2hHomeWins", "h2hHomeLosses", "h2hAwayWins", "h2hAwayLosses", "h2hDraws", "h2hAvgGoals", "h2hMatches"])
    count = 0
    for index in range(5729, len(copy)-1):
        row = copy.iloc[index]
        head2head = get_head2head(row.id)
        count = index
        print(f"Progress: {index}")
        print(head2head)
        if head2head.get("numberOfMatches"):
            copy.loc[index, "h2hHomeWins"] = head2head["homeTeam"]["wins"]
            copy.loc[index, "h2hHomeLosses"] = -head2head["homeTeam"]["losses"]
            copy.loc[index, "h2hAwayWins"] = head2head["homeTeam"]["wins"]
            copy.loc[index, "h2hAwayLosses"] = -head2head["homeTeam"]["wins"]
            copy.loc[index, "h2hDraws"] = head2head["homeTeam"]["draws"]
            copy.loc[index, "h2hAvgGoals"] = head2head["totalGoals"]
            copy.loc[index, "h2hMatches"] = head2head["numberOfMatches"]
            copy2 = pd.concat([copy2, copy.iloc[[index]]], ignore_index=True)
except:
    print(traceback.format_exc())
    copy2.to_csv(f"../relative_datasets/raw/{MODEL_NAME}-{MODEL_VERSION}_{count}_copy.csv")
copy2.to_csv(f"../relative_datasets/raw/{MODEL_NAME}-{MODEL_VERSION}_{count}_copy.csv")

In [137]:
df1 = pd.read_csv("../relative_datasets/raw/all_leagues-v1.0_628_copy.csv")
df2 = pd.read_csv("../relative_datasets/raw/all_leagues-v1.0_5668_copy.csv")
df3 = pd.read_csv("../relative_datasets/raw/all_leagues-v1.0_61_copy.csv")
df4 = pd.read_csv("../relative_datasets/raw/all_leagues-v1.0_7205_copy.csv")
dataframe = pd.concat([df1, df2, df3, df4]).drop_duplicates().dropna().reset_index(drop=True).drop(["Unnamed: 0"], axis=1)
dataframe

,id,day,month,year,weekday,hour,minute,matchday,status,leagueName,...,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals,h2hMatches
0,441764,5,5,2024,7,10,15,32.0,FINISHED,Eredivisie,...,2.0,4.0,2.0,4.0,0.0,4.0,-4.0,1.0,14.0,5.0
1,441766,5,5,2024,7,10,15,32.0,FINISHED,Eredivisie,...,2.0,0.0,5.0,2.0,-3.0,2.0,-2.0,0.0,14.0,5.0
2,444594,5,5,2024,7,10,30,35.0,FINISHED,Serie A,...,0.0,1.0,1.0,0.0,-1.0,0.0,0.0,4.0,11.0,5.0
3,441767,5,5,2024,7,12,30,32.0,FINISHED,Eredivisie,...,1.0,1.0,4.0,0.0,-4.0,0.0,0.0,1.0,18.0,5.0
4,441765,5,5,2024,7,12,30,32.0,FINISHED,Eredivisie,...,0.0,1.0,0.0,2.0,-1.0,2.0,-2.0,2.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7511,285444,18,6,2021,5,13,0,2.0,FINISHED,European Championship,...,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0,1.0
7512,285438,18,6,2021,5,16,0,2.0,FINISHED,European Championship,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0
7513,285439,18,6,2021,5,19,0,2.0,FINISHED,European Championship,...,0.0,0.0,0.0,2.0,0.0,2.0,-2.0,2.0,9.0,4.0
7514,285450,19,6,2021,6,13,0,2.0,FINISHED,European Championship,...,0.0,1.0,1.0,0.0,-2.0,0.0,0.0,1.0,9.0,3.0


In [138]:
# Clean the data and remove duplicated values in process of combining datasets
dataframe = dataframe.sort_values("id").reset_index(drop=True)
duplicated = dataframe.id.duplicated(keep="first")
for duplicate_index in duplicated.loc[duplicated == True].index:
    dataframe = dataframe.drop(index=duplicate_index)
dataframe = dataframe.reset_index(drop=True)
dataframe

,id,day,month,year,weekday,hour,minute,matchday,status,leagueName,...,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals,h2hMatches
0,285418,11,6,2021,5,19,0,1.0,FINISHED,European Championship,...,0.0,0.0,3.0,0.0,-2.0,0.0,0.0,0.0,6.0,2.0
1,285419,12,6,2021,6,13,0,1.0,FINISHED,European Championship,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0
2,285420,16,6,2021,3,16,0,2.0,FINISHED,European Championship,...,1.0,0.0,2.0,1.0,-1.0,1.0,-1.0,1.0,6.0,3.0
3,285421,16,6,2021,3,19,0,2.0,FINISHED,European Championship,...,0.0,3.0,0.0,2.0,-1.0,2.0,-2.0,2.0,13.0,5.0
4,285422,20,6,2021,7,16,0,3.0,FINISHED,European Championship,...,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,494884,25,4,2024,4,0,30,3.0,FINISHED,Copa Libertadores,...,1.0,1.0,2.0,0.0,-2.0,0.0,0.0,0.0,5.0,2.0
6925,494885,8,5,2024,3,0,0,4.0,FINISHED,Copa Libertadores,...,0.0,1.0,1.0,0.0,-1.0,0.0,0.0,1.0,5.0,2.0
6926,494886,8,5,2024,3,0,0,4.0,FINISHED,Copa Libertadores,...,2.0,2.0,2.0,0.0,-1.0,0.0,0.0,1.0,6.0,2.0
6927,495747,12,5,2024,7,11,0,1.0,FINISHED,Championship,...,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,1.0,12.0,5.0


# MANIPULATIONS

In [139]:
# REMOVE INPLAY MATCHES
dataframe = dataframe.loc[dataframe.status == "FINISHED"].reset_index(drop=True)
dataframe

,id,day,month,year,weekday,hour,minute,matchday,status,leagueName,...,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals,h2hMatches
0,285418,11,6,2021,5,19,0,1.0,FINISHED,European Championship,...,0.0,0.0,3.0,0.0,-2.0,0.0,0.0,0.0,6.0,2.0
1,285419,12,6,2021,6,13,0,1.0,FINISHED,European Championship,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0
2,285420,16,6,2021,3,16,0,2.0,FINISHED,European Championship,...,1.0,0.0,2.0,1.0,-1.0,1.0,-1.0,1.0,6.0,3.0
3,285421,16,6,2021,3,19,0,2.0,FINISHED,European Championship,...,0.0,3.0,0.0,2.0,-1.0,2.0,-2.0,2.0,13.0,5.0
4,285422,20,6,2021,7,16,0,3.0,FINISHED,European Championship,...,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,494884,25,4,2024,4,0,30,3.0,FINISHED,Copa Libertadores,...,1.0,1.0,2.0,0.0,-2.0,0.0,0.0,0.0,5.0,2.0
6920,494885,8,5,2024,3,0,0,4.0,FINISHED,Copa Libertadores,...,0.0,1.0,1.0,0.0,-1.0,0.0,0.0,1.0,5.0,2.0
6921,494886,8,5,2024,3,0,0,4.0,FINISHED,Copa Libertadores,...,2.0,2.0,2.0,0.0,-1.0,0.0,0.0,1.0,6.0,2.0
6922,495747,12,5,2024,7,11,0,1.0,FINISHED,Championship,...,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,1.0,12.0,5.0


In [140]:
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe[["id", "day", "month", "year", "weekday", "hour", "minute", "matchday",
                      "leagueType", "homeId", "awayId", "scoreHomeHt", "scoreAwayHt",
                      "scoreHomeFt", "scoreAwayFt", "h2hHomeWins", "h2hHomeLosses", "h2hAwayWins",
                      "h2hAwayLosses", "h2hDraws", "h2hAvgGoals", "h2hMatches"]]
dataframe.leagueType = (dataframe.leagueType == "CUP").astype(float)
dataframe

,id,day,month,year,weekday,hour,minute,matchday,leagueType,homeId,...,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals,h2hMatches
0,285418,11,6,2021,5,19,0,1.0,1.0,803,...,0.0,0.0,3.0,0.0,-2.0,0.0,0.0,0.0,6.0,2.0
1,285419,12,6,2021,6,13,0,1.0,1.0,833,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0
2,285420,16,6,2021,3,16,0,2.0,1.0,803,...,1.0,0.0,2.0,1.0,-1.0,1.0,-1.0,1.0,6.0,3.0
3,285421,16,6,2021,3,19,0,2.0,1.0,784,...,0.0,3.0,0.0,2.0,-1.0,2.0,-2.0,2.0,13.0,5.0
4,285422,20,6,2021,7,16,0,3.0,1.0,784,...,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,494884,25,4,2024,4,0,30,3.0,1.0,9379,...,1.0,1.0,2.0,0.0,-2.0,0.0,0.0,0.0,5.0,2.0
6920,494885,8,5,2024,3,0,0,4.0,1.0,5171,...,0.0,1.0,1.0,0.0,-1.0,0.0,0.0,1.0,5.0,2.0
6921,494886,8,5,2024,3,0,0,4.0,1.0,7055,...,2.0,2.0,2.0,0.0,-1.0,0.0,0.0,1.0,6.0,2.0
6922,495747,12,5,2024,7,11,0,1.0,0.0,68,...,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,1.0,12.0,5.0


In [147]:
dataframe = pd.read_csv(f"../relative_datasets/raw/{MODEL_NAME}-{MODEL_VERSION}-final.csv")
dataframe

,id,day,month,year,weekday,hour,minute,matchday,leagueType,homeId,...,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals,h2hMatches
0,285418,11,6,2021,5,19,0,1.0,1.0,803,...,0.0,0.0,3.0,0.0,-2.0,0.0,0.0,0.0,6.0,2.0
1,285419,12,6,2021,6,13,0,1.0,1.0,833,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0
2,285420,16,6,2021,3,16,0,2.0,1.0,803,...,1.0,0.0,2.0,1.0,-1.0,1.0,-1.0,1.0,6.0,3.0
3,285421,16,6,2021,3,19,0,2.0,1.0,784,...,0.0,3.0,0.0,2.0,-1.0,2.0,-2.0,2.0,13.0,5.0
4,285422,20,6,2021,7,16,0,3.0,1.0,784,...,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,494884,25,4,2024,4,0,30,3.0,1.0,9379,...,1.0,1.0,2.0,0.0,-2.0,0.0,0.0,0.0,5.0,2.0
6920,494885,8,5,2024,3,0,0,4.0,1.0,5171,...,0.0,1.0,1.0,0.0,-1.0,0.0,0.0,1.0,5.0,2.0
6921,494886,8,5,2024,3,0,0,4.0,1.0,7055,...,2.0,2.0,2.0,0.0,-1.0,0.0,0.0,1.0,6.0,2.0
6922,495747,12,5,2024,7,11,0,1.0,0.0,68,...,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,1.0,12.0,5.0


In [156]:
dataframe = dataframe[dataframe.scoreHomeFt != dataframe.scoreAwayFt]
dataframe["target"] = (dataframe.scoreHomeFt > dataframe.scoreAwayFt).astype(float)
dataframe = dataframe.astype(float)
dataframe

,id,day,month,year,weekday,hour,minute,matchday,leagueType,homeId,...,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals,target
0,285418.0,11.0,6.0,2021.0,5.0,19.0,0.0,1.0,1.0,803.0,...,0.0,0.0,3.0,0.0,-2.0,0.0,0.0,0.0,3.0,0.0
2,285420.0,16.0,6.0,2021.0,3.0,16.0,0.0,2.0,1.0,803.0,...,1.0,0.0,2.0,1.0,-1.0,1.0,-1.0,1.0,2.0,0.0
3,285421.0,16.0,6.0,2021.0,3.0,19.0,0.0,2.0,1.0,784.0,...,0.0,3.0,0.0,2.0,-1.0,2.0,-2.0,2.0,2.6,1.0
4,285422.0,20.0,6.0,2021.0,7.0,16.0,0.0,3.0,1.0,784.0,...,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0,1.0
5,285423.0,20.0,6.0,2021.0,7.0,16.0,0.0,3.0,1.0,788.0,...,0.0,3.0,1.0,1.0,-1.0,1.0,-1.0,0.0,3.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6915,494880.0,3.0,4.0,2024.0,3.0,22.0,0.0,1.0,1.0,7055.0,...,0.0,2.0,0.0,1.0,0.0,1.0,-1.0,0.0,2.0,1.0
6916,494881.0,10.0,4.0,2024.0,3.0,0.0,0.0,2.0,1.0,9379.0,...,0.0,3.0,0.0,1.0,0.0,1.0,-1.0,1.0,2.5,1.0
6917,494882.0,12.0,4.0,2024.0,5.0,0.0,0.0,2.0,1.0,6667.0,...,0.0,2.0,0.0,1.0,0.0,1.0,-1.0,1.0,3.0,1.0
6918,494883.0,24.0,4.0,2024.0,3.0,22.0,0.0,3.0,1.0,7055.0,...,1.0,2.0,1.0,1.0,0.0,1.0,-1.0,0.0,3.0,1.0


In [152]:
dataframe.h2hAvgGoals = dataframe.h2hAvgGoals/dataframe.h2hMatches
dataframe

,id,day,month,year,weekday,hour,minute,matchday,leagueType,homeId,...,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals,h2hMatches
0,285418.0,11.0,6.0,2021.0,5.0,19.0,0.0,1.0,1.0,803.0,...,0.0,0.0,3.0,0.0,-2.0,0.0,0.0,0.0,3.0,2.0
2,285420.0,16.0,6.0,2021.0,3.0,16.0,0.0,2.0,1.0,803.0,...,1.0,0.0,2.0,1.0,-1.0,1.0,-1.0,1.0,2.0,3.0
3,285421.0,16.0,6.0,2021.0,3.0,19.0,0.0,2.0,1.0,784.0,...,0.0,3.0,0.0,2.0,-1.0,2.0,-2.0,2.0,2.6,5.0
4,285422.0,20.0,6.0,2021.0,7.0,16.0,0.0,3.0,1.0,784.0,...,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0,1.0
5,285423.0,20.0,6.0,2021.0,7.0,16.0,0.0,3.0,1.0,788.0,...,0.0,3.0,1.0,1.0,-1.0,1.0,-1.0,0.0,3.5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6915,494880.0,3.0,4.0,2024.0,3.0,22.0,0.0,1.0,1.0,7055.0,...,0.0,2.0,0.0,1.0,0.0,1.0,-1.0,0.0,2.0,1.0
6916,494881.0,10.0,4.0,2024.0,3.0,0.0,0.0,2.0,1.0,9379.0,...,0.0,3.0,0.0,1.0,0.0,1.0,-1.0,1.0,2.5,2.0
6917,494882.0,12.0,4.0,2024.0,5.0,0.0,0.0,2.0,1.0,6667.0,...,0.0,2.0,0.0,1.0,0.0,1.0,-1.0,1.0,3.0,2.0
6918,494883.0,24.0,4.0,2024.0,3.0,22.0,0.0,3.0,1.0,7055.0,...,1.0,2.0,1.0,1.0,0.0,1.0,-1.0,0.0,3.0,1.0


In [154]:
del dataframe["h2hMatches"]
dataframe

,id,day,month,year,weekday,hour,minute,matchday,leagueType,homeId,...,scoreHomeHt,scoreAwayHt,scoreHomeFt,scoreAwayFt,h2hHomeWins,h2hHomeLosses,h2hAwayWins,h2hAwayLosses,h2hDraws,h2hAvgGoals
0,285418.0,11.0,6.0,2021.0,5.0,19.0,0.0,1.0,1.0,803.0,...,0.0,0.0,0.0,3.0,0.0,-2.0,0.0,0.0,0.0,3.0
2,285420.0,16.0,6.0,2021.0,3.0,16.0,0.0,2.0,1.0,803.0,...,0.0,1.0,0.0,2.0,1.0,-1.0,1.0,-1.0,1.0,2.0
3,285421.0,16.0,6.0,2021.0,3.0,19.0,0.0,2.0,1.0,784.0,...,1.0,0.0,3.0,0.0,2.0,-1.0,2.0,-2.0,2.0,2.6
4,285422.0,20.0,6.0,2021.0,7.0,16.0,0.0,3.0,1.0,784.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,1.0
5,285423.0,20.0,6.0,2021.0,7.0,16.0,0.0,3.0,1.0,788.0,...,2.0,0.0,3.0,1.0,1.0,-1.0,1.0,-1.0,0.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6915,494880.0,3.0,4.0,2024.0,3.0,22.0,0.0,1.0,1.0,7055.0,...,1.0,0.0,2.0,0.0,1.0,0.0,1.0,-1.0,0.0,2.0
6916,494881.0,10.0,4.0,2024.0,3.0,0.0,0.0,2.0,1.0,9379.0,...,2.0,0.0,3.0,0.0,1.0,0.0,1.0,-1.0,1.0,2.5
6917,494882.0,12.0,4.0,2024.0,5.0,0.0,0.0,2.0,1.0,6667.0,...,1.0,0.0,2.0,0.0,1.0,0.0,1.0,-1.0,1.0,3.0
6918,494883.0,24.0,4.0,2024.0,3.0,22.0,0.0,3.0,1.0,7055.0,...,2.0,1.0,2.0,1.0,1.0,0.0,1.0,-1.0,0.0,3.0


# SPORTS FEATURE ENGINEERING

In [ ]:
# CONVERT DATES TO DATETIME TO EXTRACT MORE FEATURES FROM THEM
dataframe[["HourOfDay", "MinuteOfHour"]] = dataframe["Time"].str.split(":", expand=True)
del dataframe["Time"]

dataframe.index = pd.to_datetime(dataframe.pop("Date"), format="mixed")
dataframe["DayOfWeek"] = dataframe.index.day_of_week
dataframe["DayOfYear"] = dataframe.index.day_of_year
dataframe = dataframe.sort_index()
dataframe

,AC,AS,Avg<2.5,Avg>2.5,AvgA,AvgD,AvgH,AwayTeam,Div,FTAG,FTHG,FTR,HC,HS,HTR,HomeTeam,HourOfDay,MinuteOfHour,DayOfWeek,DayOfYear
Date,,,,,,,,,,,,,,,,,,,,
2019-01-09,7,8,1.66,2.25,3.65,3.29,2.17,Wolves,E0,2,3,H,7,15,H,Everton,14,00,2,9
2019-01-09,6,13,2.40,1.57,2.90,3.63,2.38,Tottenham,E0,2,2,D,11,26,A,Arsenal,16,30,2,9
2019-01-10,3,12,1.96,1.85,2.54,3.42,2.75,Sheffield Weds,E1,0,1,H,8,8,D,Hull,19,45,3,10
2019-01-10,3,10,1.89,1.92,3.56,3.46,2.08,Nott'm Forest,E1,1,1,D,11,21,D,Blackburn,19,45,3,10
2019-01-10,6,10,2.15,1.70,4.42,3.75,1.80,West Brom,E1,0,1,H,7,19,H,Leeds,19,45,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03,2,5,1.87,1.89,4.94,3.80,1.66,Reading,E2,1,2,H,7,18,D,Derby,19,45,1,338
2024-12-03,9,24,2.07,1.73,1.68,3.87,4.68,Barnsley,E2,3,2,A,6,7,D,Carlisle,19,45,1,338
2024-12-03,4,12,2.04,1.75,2.38,3.52,2.87,Middlesbrough,E1,1,0,A,3,9,A,Birmingham,19,45,1,338


In [ ]:
# ENCODING THE TEAMS
team_encodings = dict()
div_encodings = {"E0": 0, "E1": 1, "E2": 2}
result_encoding = dict(H=0, A=1, D=-1)
encodings_counter = 0
div_counter = 0
for index, row in dataframe.iterrows():
    if team_encodings.get(row.HomeTeam) is None:
        team_encodings[row.HomeTeam] = encodings_counter
        encodings_counter += 1

    if team_encodings.get(row.AwayTeam) is None:
        team_encodings[row.AwayTeam] = encodings_counter
        encodings_counter += 1

    if div_encodings.get(row.Div) is None:
        div_encodings[row.Div] = encodings_counter
        encodings_counter += 1

    dataframe.loc[index, 'HomeTeam'] = team_encodings[row.HomeTeam]
    dataframe.loc[index, 'AwayTeam'] = team_encodings[row.AwayTeam]
    dataframe.loc[index, 'Div'] = div_encodings[row.Div]
    dataframe.loc[index, 'FTR'] = result_encoding[row.FTR]
    dataframe.loc[index, 'HTR'] = result_encoding[row.HTR]
dataframe = dataframe.astype(float)
dataframe

,AC,AS,Avg<2.5,Avg>2.5,AvgA,AvgD,AvgH,AwayTeam,Div,FTAG,FTHG,FTR,HC,HS,HTR,HomeTeam,HourOfDay,MinuteOfHour,DayOfWeek,DayOfYear
Date,,,,,,,,,,,,,,,,,,,,
2019-01-09,7.0,8.0,1.66,2.25,3.65,3.29,2.17,3.0,0.0,2.0,3.0,-1.0,7.0,15.0,1.0,2.0,14.0,0.0,2.0,9.0
2019-01-09,6.0,13.0,2.40,1.57,2.90,3.63,2.38,3.0,0.0,2.0,2.0,-1.0,11.0,26.0,1.0,2.0,16.0,30.0,2.0,9.0
2019-01-10,3.0,12.0,1.96,1.85,2.54,3.42,2.75,17.0,1.0,0.0,1.0,0.0,8.0,8.0,-1.0,16.0,19.0,45.0,3.0,10.0
2019-01-10,3.0,10.0,1.89,1.92,3.56,3.46,2.08,17.0,1.0,1.0,1.0,0.0,11.0,21.0,-1.0,16.0,19.0,45.0,3.0,10.0
2019-01-10,6.0,10.0,2.15,1.70,4.42,3.75,1.80,17.0,1.0,0.0,1.0,0.0,7.0,19.0,-1.0,16.0,19.0,45.0,3.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03,2.0,5.0,1.87,1.89,4.94,3.80,1.66,75.0,2.0,1.0,2.0,1.0,7.0,18.0,1.0,54.0,19.0,45.0,1.0,338.0
2024-12-03,9.0,24.0,2.07,1.73,1.68,3.87,4.68,75.0,2.0,3.0,2.0,1.0,6.0,7.0,1.0,54.0,19.0,45.0,1.0,338.0
2024-12-03,4.0,12.0,2.04,1.75,2.38,3.52,2.87,75.0,2.0,1.0,0.0,1.0,3.0,9.0,1.0,54.0,19.0,45.0,1.0,338.0


In [ ]:
dataframe["target"] = dataframe.pop("FTR")
dataframe = dataframe[["HomeTeam", "AwayTeam", "AvgH", "AvgD", "AvgA", "DayOfYear", "DayOfWeek", "HourOfDay", "MinuteOfHour", "target"]]
dataframe

,HomeTeam,AwayTeam,AvgH,AvgD,AvgA,DayOfYear,DayOfWeek,HourOfDay,MinuteOfHour,target
Date,,,,,,,,,,
2019-01-09,2.0,3.0,2.17,3.29,3.65,9.0,2.0,14.0,0.0,-1.0
2019-01-09,2.0,3.0,2.38,3.63,2.90,9.0,2.0,16.0,30.0,-1.0
2019-01-10,16.0,17.0,2.75,3.42,2.54,10.0,3.0,19.0,45.0,0.0
2019-01-10,16.0,17.0,2.08,3.46,3.56,10.0,3.0,19.0,45.0,0.0
2019-01-10,16.0,17.0,1.80,3.75,4.42,10.0,3.0,19.0,45.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2024-12-03,54.0,75.0,1.66,3.80,4.94,338.0,1.0,19.0,45.0,1.0
2024-12-03,54.0,75.0,4.68,3.87,1.68,338.0,1.0,19.0,45.0,1.0
2024-12-03,54.0,75.0,2.87,3.52,2.38,338.0,1.0,19.0,45.0,1.0


In [ ]:
dataframe = dataframe.loc[dataframe.target != -1]
dataframe

,HomeTeam,AwayTeam,AvgH,AvgD,AvgA,DayOfYear,DayOfWeek,HourOfDay,MinuteOfHour,target
Date,,,,,,,,,,
2019-01-10,16.0,17.0,2.75,3.42,2.54,10.0,3.0,19.0,45.0,0.0
2019-01-10,16.0,17.0,2.08,3.46,3.56,10.0,3.0,19.0,45.0,0.0
2019-01-10,16.0,17.0,1.80,3.75,4.42,10.0,3.0,19.0,45.0,0.0
2019-01-10,16.0,17.0,2.66,3.20,2.75,10.0,3.0,19.0,45.0,0.0
2019-01-10,16.0,17.0,1.79,3.57,4.73,10.0,3.0,20.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2024-12-03,54.0,75.0,1.66,3.80,4.94,338.0,1.0,19.0,45.0,1.0
2024-12-03,54.0,75.0,4.68,3.87,1.68,338.0,1.0,19.0,45.0,1.0
2024-12-03,54.0,75.0,2.87,3.52,2.38,338.0,1.0,19.0,45.0,1.0


In [ ]:
# ADD FINANCIAL INDICATORS AND TARGET CLASS
dataframe = append_technical_indicators(dataframe, add_outcomes=True)

# DATA SPLITTING

In [157]:
# TRAIN AND TEST DATASET SPLIT
targets = dataframe.pop("target")
X, X_test, Y, Y_test = train_test_split(dataframe, targets, test_size=0.2, random_state=False)
f"""TRAIN: {X.shape}, TEST: {X_test.shape}"""

'TRAIN: (4168, 21), TEST: (1042, 21)'

In [159]:
# SAVING THE TRAIN TEST SPLITS TO A FINALS FOLDER
X.join(Y).to_csv(f"../relative_datasets/cleaned/{MODEL_NAME}-{MODEL_VERSION}-train-set.csv", index=False)
X_test.join(Y_test).to_csv(f"../relative_datasets/cleaned/{MODEL_NAME}-{MODEL_VERSION}-test-set.csv", index=False)

In [163]:
import numpy as np
np.where(dataframe.homeId.unique().astype(int) == )

(array([93]),)

# SAVING THE SOCCER LABELS

In [ ]:
teams_encoding_dataframe = pd.DataFrame(
                [[team_name, team_encodings[team_name]] for team_name in team_encodings.keys()],
                columns=["TeamName", "Encoding"])
teams_encoding_dataframe.to_csv(f"../relative_datasets/cleaned/{MODEL_NAME}-{MODEL_VERSION}-team-labels-set.csv", index=False)

In [ ]:
# UPDATING THE GITHUB OF THE DATASETS
!git add -A
!git commit -m "updates: {MODEL_NAME}-{MODEL_VERSION}: bot commit"
!git push

[main 15598e9] updates: eng_prem_league-v1.0: bot commit
 3 files changed, 19 insertions(+), 10 deletions(-)
 create mode 100644 relative_datasets/cleaned/mapping_labels_bank.json
Enumerating objects: 14, done.
Counting objects: 100% (14/14), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 1.03 KiB | 1.03 MiB/s, done.
Total 8 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/lebyanelm/neural-trained-models.git
   13805c1..15598e9  main -> main
